In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import xarray as xr
from scipy.interpolate import interp1d
from netCDF4 import Dataset

In [15]:
thing = xr.open_dataset('/home/disk/pna2/aodhan/ERA5_hourly_data/december_2007_ERA5.nc')

thing

<xarray.Dataset>
Dimensions:    (latitude: 721, level: 19, longitude: 1440, time: 744)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.25 359.5 359.75
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * level      (level) int32 2 3 5 7 10 20 30 50 ... 175 200 225 250 300 350 400
  * time       (time) datetime64[ns] 2007-12-01 ... 2007-12-31T23:00:00
Data variables:
    t          (time, level, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2021-07-20 00:59:56 GMT by grib_to_netcdf-2.20.0: /opt/ecmw...

In [ ]:
import time 
start = time.time()
dr_versions = ['cdr', 'icdr']
cdr_satellites = ['champ', 'cosmic', 'grace', 'metop']
icdr_satellites = ['metop']
ROM_SAF_path_generic = '/home/bdc2/aodhan/ROM_SAF/www.romsaf.org/pub/{cdr_version}/profs/{satellite}/atm/'
heights = np.linspace(5,35,301)*1000
month_strings = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

ROM_SAF_temp_profiles = []
for dr in dr_versions:
    if dr == 'cdr':
        dr_string = 'cdr/v1.0'
        satellites = cdr_satellites
    elif dr == 'icdr':
        dr_string = 'icdr/v1-series'
        satellites = icdr_satellites
    for sat in satellites:
        print(sat)
        ROM_SAF_path_satellite = ROM_SAF_path_generic.format(cdr_version=dr_string, satellite=sat)
        for year in range(2001, 2023):
            print('Year: ', str(year))
            for month in month_strings:
                RO_daily_data_paths = glob.glob(ROM_SAF_path_satellite + str(year) 
                                                + '/' + str(year) + '-' + month + '*')
                if len(RO_daily_data_paths) == 0:
                    print('No data for {month} of {year}'.format(month=month, year=str(year)))
                    continue
                month_of_RO_data = []
                for day in RO_daily_data_paths:
                    datafiles_of_day = glob.glob(day + '/*')
                    
                    for datafile in datafiles_of_day:
                        if datafile[-11:] == 'non-nominal':
                            continue
                        else:
                            # extract temp and alt info from RO
                            RO = Dataset(datafile)
                            dry_temp = RO.variables['dry_temp'][0]
                            alt = RO.variables['alt_refrac'][0]
                            # above is the height according to page 51/51 at url below
                            # https://www.romsaf.org/product_documents/romsaf_pum.pdf 
                            
                            # find RO location
                            lat = np.array(RO.variables['lat_tp'])
                            lon = np.array(RO.variables['lon_tp'])
                            altm17 = np.abs(alt-17000)
                            idx_17km = np.nanargmin(altm17) # 17 Km tangent point
                            lat_17km = lat[0][idx_17km]
                            lon_17km = lon[0][idx_17km]
                            
                            # interpolate temperature profile
                            temp_interp = interp1d(alt, dry_temp, kind='cubic', bounds_error=False)
                            temp_100m_spacing = temp_interp(heights)

                            # append data
                            month_of_RO_data.append([lat_17km, lon_17km, temp_100m_spacing])
                
                # save month of data
                file_name = 'DryTempProf_{satellite}_{month}_{year}_5_35km'.format(satellite=sat, month=month, year=str(year))
                file_path = ROM_SAF_path_satellite + str(year) + '/' + file_name
                np.save(file_path, month_of_RO_data)


champ
Year:  2001
No data for 01 of 2001
No data for 02 of 2001
No data for 03 of 2001
No data for 04 of 2001
No data for 05 of 2001
No data for 06 of 2001
No data for 07 of 2001
No data for 08 of 2001
Year:  2002
Year:  2003
Year:  2004
Year:  2005
Year:  2006
Year:  2007
Year:  2008
No data for 11 of 2008
No data for 12 of 2008
Year:  2009
No data for 01 of 2009
No data for 02 of 2009
No data for 03 of 2009
No data for 04 of 2009
No data for 05 of 2009
No data for 06 of 2009
No data for 07 of 2009
No data for 08 of 2009
No data for 09 of 2009
No data for 10 of 2009
No data for 11 of 2009
No data for 12 of 2009
Year:  2010
No data for 01 of 2010
No data for 02 of 2010
No data for 03 of 2010
No data for 04 of 2010
No data for 05 of 2010
No data for 06 of 2010
No data for 07 of 2010
No data for 08 of 2010
No data for 09 of 2010
No data for 10 of 2010
No data for 11 of 2010
No data for 12 of 2010
Year:  2011
No data for 01 of 2011
No data for 02 of 2011
No data for 03 of 2011
No data for

In [98]:
import time 
start = time.time()
dr_versions = ['icdr']
cdr_satellites = ['champ', 'cosmic', 'grace', 'metop']
icdr_satellites = ['metop']
ROM_SAF_path_generic = '/home/bdc2/aodhan/ROM_SAF/www.romsaf.org/pub/{cdr_version}/profs/{satellite}/atm/'
heights = np.linspace(5,35,301)*1000
month_strings = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

ROM_SAF_temp_profiles = []
for dr in dr_versions:
    if dr == 'cdr':
        dr_string = 'cdr/v1.0'
        satellites = cdr_satellites
    elif dr == 'icdr':
        dr_string = 'icdr/v1-series'
        satellites = icdr_satellites
    for sat in satellites:
        print(sat)
        ROM_SAF_path_satellite = ROM_SAF_path_generic.format(cdr_version=dr_string, satellite=sat)
        for year in range(2001, 2023):
            print('Year: ', str(year))
            for month in month_strings:
                RO_daily_data_paths = glob.glob(ROM_SAF_path_satellite + str(year) 
                                                + '/' + str(year) + '-' + month + '*')
                if len(RO_daily_data_paths) == 0:
                    print('No data for {month} of {year}'.format(month=month, year=str(year)))
                    continue
                month_of_RO_data = []
                for day in RO_daily_data_paths:
                    datafiles_of_day = glob.glob(day + '/*')
                    
                    for datafile in datafiles_of_day:
                        if datafile[-11:] == 'non-nominal':
                            continue
                        else:
                            # extract temp and alt info from RO
                            RO = Dataset(datafile)
                            dry_temp = RO.variables['dry_temp'][0]
                            alt = RO.variables['alt_refrac'][0]
                            # above is the height according to page 51/51 at url below
                            # https://www.romsaf.org/product_documents/romsaf_pum.pdf 
                            
                            # find RO location
                            lat = np.array(RO.variables['lat_tp'])
                            lon = np.array(RO.variables['lon_tp'])
                            altm17 = np.abs(alt-17000)
                            idx_17km = np.nanargmin(altm17) # 17 Km tangent point
                            lat_17km = lat[0][idx_17km]
                            lon_17km = lon[0][idx_17km]
                            
                            # interpolate temperature profile
                            temp_interp = interp1d(alt, dry_temp, kind='cubic', bounds_error=False)
                            temp_100m_spacing = temp_interp(heights)

                            # append data
                            month_of_RO_data.append([lat_17km, lon_17km, temp_100m_spacing])
                
                # save month of data
                file_name = 'DryTempProf_{satellite}_{month}_{year}_5_35km'.format(satellite=sat, month=month, year=str(year))
                file_path = ROM_SAF_path_satellite + str(year) + '/' + file_name
                np.save(file_path, month_of_RO_data)


metop
Year:  2001
No data for 01 of 2001
No data for 02 of 2001
No data for 03 of 2001
No data for 04 of 2001
No data for 05 of 2001
No data for 06 of 2001
No data for 07 of 2001
No data for 08 of 2001
No data for 09 of 2001
No data for 10 of 2001
No data for 11 of 2001
No data for 12 of 2001
Year:  2002
No data for 01 of 2002
No data for 02 of 2002
No data for 03 of 2002
No data for 04 of 2002
No data for 05 of 2002
No data for 06 of 2002
No data for 07 of 2002
No data for 08 of 2002
No data for 09 of 2002
No data for 10 of 2002
No data for 11 of 2002
No data for 12 of 2002
Year:  2003
No data for 01 of 2003
No data for 02 of 2003
No data for 03 of 2003
No data for 04 of 2003
No data for 05 of 2003
No data for 06 of 2003
No data for 07 of 2003
No data for 08 of 2003
No data for 09 of 2003
No data for 10 of 2003
No data for 11 of 2003
No data for 12 of 2003
Year:  2004
No data for 01 of 2004
No data for 02 of 2004
No data for 03 of 2004
No data for 04 of 2004
No data for 05 of 2004
No 